# Convert video to Image

In [1]:
import cv2

In [8]:
rotate_video = True # rotate saved image 90* clockwise
frame_delta = 10 # save image for each 10 frame


In [9]:
labels = []
for name in os.listdir("videos"):
    video_name,  video_extension = name.split(".")
    labels.append(video_name)
    
    cap = cv2.VideoCapture('videos/' + video_name + video_extension)

    counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if cap.get(cv2.CAP_PROP_POS_FRAMES) % frame_delta == 0 :
            if rotate_video :
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            cv2.imwrite("dataset/images/%s_%d.jpg" % (video_name, counter), frame)
            counter += 1

    print("finish processing %d images %s..." % (counter, video_name))
    cap.release()

finish processing 99 images...


# Annotate Dataset

- Download & Install `Git` :
    - https://git-scm.com/downloads

- Clone `labelImg` repository using git
    - or just download repo as zip file.

In [ ]:
! git clone https://github.com/tzutalin/labelImg.git

- change dir to labelImg cloned repo

In [10]:
%cd labelImg

C:\Users\yunus\Desktop\dataset_from_video\images


- install dependency

In [ ]:
! conda install pyqt=5
! conda install -c anaconda lxml
! pyrcc5 -o libs/resources.py resources.qrc

- run `lableImg`

In [ ]:
! python labelImg.py

- open dir `images/` in this project

- annotate image dataset

# Split Dataset

In [23]:
split_size = 0.2 # 20% for test dataset 

In [22]:
import os
import shutil

try :
    os.mkdir("dataset/images/train")
    os.mkdir("dataset/images/test")
except :
    print("`train/` and `test/` dir already exist!")

`train/` and `test/` dir already exist!


In [25]:
all_filenames = []
for file in os.listdir("dataset/images"):
    if file.endswith(".jpg"):
        all_filenames.append(file.replace(".jpg", ""))

file_counter = {}
file_group = {}
for name in all_filenames:
    label = name.split("_")[0]
    try :
        file_counter[label] += 1
        file_group[label].append(name)
    except :
        file_counter[label] = 0
        file_group[label] = []
        
print(file_counter)
        
for label in file_counter:
    n_split = int(file_counter[label]*split_size)
    for i, name in enumerate(file_group[label]) :
        if i < n_split :
            shutil.move("dataset/images/" + name + ".jpg", "dataset/images/test")
            shutil.move("dataset/images/" + name + ".xml", "dataset/images/test")
        else :
            shutil.move("dataset/images/" + name + ".jpg", "dataset/images/train")
            shutil.move("dataset/images/" + name + ".xml", "dataset/images/train")        

# Create Lambel Map data
- label map structure :
```
item {
	id : 1
	name : "batman"
}
item {
	id : 2
	name : "ironman"
}
item {
	id : 3
	name : "xxxxxx"
}
```

In [45]:
filename = "dataset/data/object-detection.pbtxt"

with open(filename, "w") as fw:
    for i, name in enumerate(file_counter, start=1) :
        fw.write('item {\n')
        fw.write('\tid : %d\n' % i)
        fw.write('\tname : "%s"\n' % name)
        fw.write('}\n')

# Zip result

In [42]:
import zipfile

In [44]:
def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))

zipf = zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('dataset/', zipf)
zipf.close()

print("`dataset.zip` created successfully!")

# Run Faster R-CNN Training in Colab

- Open Google Colab : https://colab.research.google.com/notebooks/intro.ipynb#recent=true
- Click tab **Upload**,
- Upload `Faster_R_CNN_Training_using_Custom_Dataset.ipynb` to google colab